# 1. 결정 트리 훈련

1-a . make_moons (n_sample=1000, noise=0.4)를 사용해 데이터셋을 생성합니다.

In [2]:
from sklearn.datasets import make_moons

X,y = make_moons(n_samples=1000, noise=0.4, random_state=42)

1-b. 이를 train_test_split( )을 사용해 훈련 세트와 테스트 세트로 나눕니다.(random_state=42, train:test = 8:2 비율로)


In [60]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

1-c. DecisionTreeClassifier의 최적의 매개변수를 찾기 위해 교차 검증과 함께 그리드 탐색을 수행합니다. (GridSearchCV)

힌트 : 여러가지 max_leaf_nodes 값을 시도해보세요! (교재 pg. 113, 207 참고)

In [70]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 파라미터 정의
param_grid = {
    'max_leaf_nodes' : list(range(2,100)),
}

grid_search = GridSearchCV(
    estimator = DecisionTreeClassifier(random_state=42),
    param_grid = param_grid,
    cv=3,
    scoring='accuracy',
    verbose=1
)

grid_search.fit(X_train, y_train)

print("Best parameters:" , grid_search.best_params_)
print("Best cross-validation score", grid_search.best_score_)

Fitting 3 folds for each of 98 candidates, totalling 294 fits
Best parameters: {'max_leaf_nodes': 4}
Best cross-validation score 0.8537448490139207


In [74]:
# 최적의 매개변수 확인
grid_search.best_estimator_

DecisionTreeClassifier(max_leaf_nodes=4, random_state=42)

1-d. 찾은 매개변수를 사용해 전체 훈련 세트에 대해 모델을 훈련시키고 테스트 세트에서 성능(정확도)을 측정합니다. (대략 85~87% 정도 나옵니다.)

In [82]:
# GridSearchCV는 최적의 모델로 다시 훈련시키기 때문에 별도의 훈련이 필요없습니다.

best_tree_clf = DecisionTreeClassifier(
    max_leaf_nodes = grid_search.best_params_['max_leaf_nodes'],
    random_state=42
)
best_tree_clf.fit(X_train, y_train)

from sklearn.metrics import accuracy_score

y_pred = best_tree_clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print(f"테스트 정확도: {test_accuracy:.3f}")

테스트 정확도: 0.855


# 2. 랜덤포레스트

In [122]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

# !wget 'https://bit.ly/3i4n1QB'

# import zipfile
# with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
#    existing_zip.extractall('data')

In [110]:
# 라이브러리 불러오기

import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

In [128]:
train = pd.read_csv('./train.csv')

# Scaling
scaler = MinMaxScaler()
scaler.fit(train[['fixed acidity']])
train['Scaled fixed acidity'] = scaler.transform(train[['fixed acidity']])

# Encoding
encoder = OneHotEncoder()
encoder.fit(train[['type']])
onehot = encoder.transform(train[['type']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns = encoder.get_feature_names_out()
train = pd.concat([train, onehot], axis = 1)
train = train.drop(columns = ['type'])
train.head()

,index,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,Scaled fixed acidity,type_red,type_white
0,0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,0.148760,0.0,1.0
1,1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,0.413223,1.0,0.0
2,2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,0.338843,0.0,1.0
3,3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,0.264463,0.0,1.0
4,4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,0.330579,0.0,1.0


2-a. 랜덤포레스트 분류 모형을 불러옵니다.

In [130]:
from sklearn.ensemble import RandomForestClassifier

2-b. 랜덤포레스트 분류 모형을 "random_classifier"라는 변수에 저장합니다.

In [133]:
random_classifier =  RandomForestClassifier()

2-c. "X"라는 변수에 train의 "quality" 피쳐를 제거하고 저장합니다.

In [136]:
X = train.drop('quality', axis=1)

2-d. "y"라는 변수에 정답인 train의 "quality" 피쳐를 저장합니다.

In [151]:
y = train['quality']

2-e. "random_classifier"를 X와 y를 이용해 학습시켜보세요.

In [158]:
random_classifier.fit(X, y)

RandomForestClassifier()